In [1]:
import sys
from FDMBuilder.FDMTable import FDMTable
from FDMTable import *
from FDMDataset import *

ModuleNotFoundError: No module named 'FDM_helpers'

## TO DO:

* PRIORITY: Work our what happens if you re-run .build() on a dataset
* re-running .build() needs "outside_obs" tables to be added back to src tables - TEST THIS


* THEN: build FDM datasets from just dataset_id:
    + summarise tables in dataset init i.e. which are ready for FDM build, which arent, any FDM tables that will be overwritten, any src tables that will be re-combined
    * think about "verbose" functionality of FDMTables - silence init-messages (not required anyways as tables will be initialised from tables in FDM dataset) - "check_build" function
    
    
* add check and warning for Null datetimes when adding start/end dates
* add .drop() method to table
* raw input functionality - copying src tables? overwriting existing tables?
* build data dict function
* ask about requirement to have a raw version of FDM_tables at all times? - read only source?

## Testing:

In [ ]:
build_test_environment()

In [ ]:
table_1 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_1",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_1.quick_build(event_start_date_cols="start_date",
                     event_start_date_format="YMD",
                     event_end_date_cols="end_date",
                     event_end_date_format="YMD")

In [ ]:
table_2 = FDMTable(
    source_table_full_id = f"{PROJECT}.CY_TESTS_SRC.src_table_2",
    dataset_id = "CY_TESTS_FDM" 
)

In [ ]:
table_2.quick_build(
    event_start_date_cols=["start_year", "start_month", "start_day"], 
    event_start_date_format="YMD", 
    event_end_date_cols=["end_year", "end_month", "end_day"], 
    event_end_date_format="YMD"
)

In [ ]:
table_3 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_3",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_3.quick_build(
    event_start_date_cols=["year", "month", "15"], 
    event_start_date_format="YMD", 
)

In [ ]:
table_4 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_4",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_4.quick_build(
    event_start_date_cols="start_date_string",
    event_start_date_format="MDY", 
    event_end_date_cols="end_date_string",
    event_end_date_format="DMY"
)

In [ ]:
table_5 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_5",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_5.copy_table_to_dataset()

In [ ]:
table_5.head()

In [ ]:
table_5.add_column("SPLIT(date_string, '-')[OFFSET(0)] AS year")
table_5.add_column("SPLIT(date_string, '-')[OFFSET(1)] AS month")
table_5.head()

In [ ]:
table_5.quick_build(
    event_start_date_cols=["year", "month", "15"],
    event_start_date_format="YMD", 
)

In [ ]:
table_6 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_6",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_6.copy_table_to_dataset()

In [ ]:
table_6.head()

In [ ]:
table_6.quick_build(
    event_start_date_cols="date_string",
    event_start_date_format="DMY",
)

In [ ]:
dataset = FDMDataset(
    dataset_id = "CY_TESTS_FDM"
)

In [ ]:
dataset.build()

In [ ]:
blah = "got_money_mo_problems"
blah[:-9]